<a href="https://colab.research.google.com/github/socd06/databricks-language-hackathon/blob/main/June5_Dolly_3B_PDF_QA_Chain_with_instruct_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate einops Xformers langchain InstructorEmbedding sentence-transformers chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.6/86.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 k

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install unstructured pandoc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 61.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.0/127.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 kB 

In [31]:
# CLEAR CUDA
import gc
torch.cuda.empty_cache()
gc.collect()


8922

In [32]:
import torch
# original
from instruct_pipeline import InstructionTextGenerationPipeline
# mine
# from instruct_pipeline_as_ai_ethicist import InstructionTextPipelineAsAIEthicist
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "databricks/dolly-v2-3b"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)

generate_text = InstructionTextGenerationPipeline(task='text-generation',
                                                  model=model, 
                                                  tokenizer=tokenizer)


In [33]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
import torch

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)

In [28]:
# TODO: Check if input is checked somewhere, otherwise check something like this:

# Your output sequence
output_sequence = "This is an example output sequence."

# Tokenize the output sequence
tokenized_output = tokenizer.encode(output_sequence, return_tensors="pt")

# Count the number of tokens in the tokenized sequence
num_tokens = tokenized_output.shape[1]

print(f"Number of tokens in the output sequence: {num_tokens}")

Number of tokens in the output sequence: 7


# Test LLM Chain

In [34]:
question = 'Who was Dolly the sheep?'
llm_chain.run(question)

'nimal that was the first successful cloned mammal. Dolly was created using the process of somatic cell nuclear transfer (SCNT), which was pioneered in 1997 by Japanese researchers.'

In [35]:
question = 'what are the colors of the rainbow?'
llm_chain.run(question)

're: red, orange, yellow, green, blue, indigo and violet. These are pure colors and cannot be mixed.'

# Prepare Documents


In [38]:
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import os
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [ ]:
# Load and process the PDF files
loader = DirectoryLoader('/content/docs/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [ ]:
# Verify we were able to load the docs
documents[0:2]

[Document(page_content='  \n \n  \n   \n  BLUEPRINT FOR AN \nAI B ILL OF \nRIGHTS \nMAKING AUTOMATED \nSYSTEMS WORK FOR \nTHE AMERICAN PEOPLE \nOCTOBER 2022 \n', metadata={'source': '/content/docs/Blueprint-for-an-AI-Bill-of-Rights.pdf', 'page': 0}),
 Document(page_content=' \n \n  \n  \n \n \n \n \n \n \n \n About this Document \nThe Blueprint for an AI Bill of Rights: Making Automated Systems Work for the American People was \npublished by the White House Office of Science and Technology Policy in October 2022. This framework was \nreleased one year after OSTP announced  the launch of a process to develop “a bill of rights for an AI-powered \nworld.” Its release follows a year of public engagement to inform this initiative. The framework is available \nonline at: https://www.whitehouse.gov/ostp/ai-bill-of-rights \nAbout the Office of Science and Technology Policy \nThe Office of Science and Technology Policy (OSTP)  was established by the National Science and Technology  \nPolicy, Or

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Download HuggingFace Embeddings
Check [MTEB English Leaderboard](https://huggingface.co/spaces/mteb/leaderboard) to make sure you download embeddings with good performance

In [ ]:
# Choose one of the top performers from the MTEB English Leaderboard

from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# top #2 when task = Retrieval June 2023 for under ~500 MB
model_name = "intfloat/e5-base-v2" 

hf = HuggingFaceEmbeddings(model_name=model_name)

# Make a Vector Database

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = hf 

vectordb = Chroma.from_documents(documents=documents, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

# and a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("What is the AI BILL OF RIGHTS?")

# Test the basic retriever

In [ ]:
docs[0]

Document(page_content=' \n  ABOUT THIS  FRAMEWORK\nThe Blueprint for an AI Bill of Rights is a set of five principles and associated practices to help guide the \ndesign, use, and deployment of automated systems to protect the rights of the American public in the age of \nartificial intel-ligence. Developed through extensive consultation with the American public, these principles are \na blueprint for building and deploying automated systems that are aligned with democratic values and protect \ncivil rights, civil liberties, and privacy. The Blueprint for an AI Bill of Rights includes this Foreword, the five \nprinciples, notes on Applying the The Blueprint for an AI Bill of Rights, and a Technical Companion that gives \nconcrete steps that can be taken by many kinds of organizations—from governments at all levels to companies of \nall sizes—to uphold these values. Experts from across the private sector, governments, and international \nconsortia have published principles and framework

In [ ]:
docs = retriever.get_relevant_documents("What does AI RMF mean?")

# Make a proper Question Retrieval chain

In [ ]:
# use map_reduce to prevent token mismatch errors?
# possible issue  due to batch size?
qa_chain = RetrievalQA.from_chain_type(llm=hf_pipeline, 
                                       chain_type="map_reduce", 
                                       retriever=retriever, 
                                       return_source_documents=True,
                                       verbose=True)

qa_chain.combine_documents_chain.llm_chain.prompt.template = '''
You are an AI Ethicists. 
Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always answer with unbiased ethical and safe advise.
----------------
{context}

Question: {question}
Helpful Answer:'''

In [ ]:
def trim_string(input_string):
    input_string = str(input_string)
    trim_index = input_string.find("### Human:")
    if trim_index != -1:  # If the phrase is found
        return input_string[:trim_index]
    else:
        return input_string  # If the phrase isn't found, return the original string

In [ ]:

## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)
    print(f'Wrapped Text is: {len(wrapped_text)} chars long')
    return wrapped_text

In [ ]:
"""
import torch

def wrap_text_preserve_newlines(text, width=110):
    # Convert the input text to a PyTorch tensor
    input_tensor = torch.tensor(text)

    # Reshape the tensor to have a single dimension
    input_tensor = input_tensor.view(-1)

    # Perform the text wrapping on the tensor
    wrapped_tensor = textwrap.wrap(input_tensor, width=width)

    # Convert the wrapped tensor back to a string
    wrapped_text = '\n'.join(wrapped_tensor)
    print(f'Wrapped Text is: {len(wrapped_text)} chars long')
    return wrapped_text
"""

# Use an Agent Tools to summarize conversation memory

In [ ]:
from transformers import load_tool
summarizer = load_tool('summarization')

In [ ]:
summarizer("""Empowering the world to develop technology through collective knowledge.
Our public platform serves 100 million people every month, making it one of the most popular websites in the world.
Our asynchronous knowledge management and collaboration offering, Stack Overflow for Teams, is transforming how people work.""")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"The Stack Overflow platform serves 100 million people every month. It is one of the most popular websites in the world. It's a knowledge management and collaboration offering, Stack overflow for Teams, is transforming how people work. It empowers people to develop technology through collective knowledge."

In [ ]:
def process_llm_response(llm_response):
    temp_resp = wrap_text_preserve_newlines(llm_response['result'])
    temp_resp = trim_string(temp_resp)
    print(temp_resp)
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])


In [ ]:
# question from AI Bill of Rights
query = "What is the AI Bill of Rights?"
llm_response = qa_chain(query)
# process_llm_response(llm_response) # this errors out sometimes



> Entering new RetrievalQA chain...


Token indices sequence length is longer than the specified maximum sequence length for this model (1935 > 1024). Running this sequence through the model will result in indexing errors


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-36-99550f8cbecc>:3 in <cell line: 3>                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:140 in __call__                 │
│                                                                                                  │
│   137 │   │   │   )                                                                              │
│   138 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   139 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 140 │   │   │   raise e                                                                        │
│   141 │   │   run_manager.on_chain_end(outputs)                                                  │
│   142 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   143                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:134 in __call__                 │
│                                                                                                  │
│   131 │   │   )                                                                                  │
│   132 │   │   try:                                                                               │
│   133 │   │   │   outputs = (                                                                    │
│ ❱ 134 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   135 │   │   │   │   if new_arg_supported                                                       │
│   136 │   │   │   │   else self._call(inputs)                                                    │
│   137 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:120 in _call       │
│                                                                                                  │
│   117 │   │   question = inputs[self.input_key]                                                  │
│   118 │   │                                                                                      │
│   119 │   │   docs = self._get_docs(question)                                                    │
│ ❱ 120 │   │   answer = self.combine_documents_chain.run(                                         │
│   121 │   │   │   input_documents=docs, question=question, callbacks=_run_manager.get_child()    │
│   122 │   │   )                                                                                  │
│   123                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:239 in run                      │
│                                                                                                  │
│   236 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   237 │   │                                                                                      │
│   238 │   │   if kwargs and not args:                                                            │
│ ❱ 239 │   │   │   return self(kwargs, callbacks=callbacks)[self.output_keys[0]]                  │
│   240 │   │                                                

In [ ]:
llm_response

In [ ]:
# Question from NIST.AI.100-1
query = "What is resilliency in AI systems?"
llm_response = qa_chain(query)
# process_llm_response(llm_response)



> Entering new RetrievalQA chain...


Token indices sequence length is longer than the specified maximum sequence length for this model (2196 > 1024). Running this sequence through the model will result in indexing errors


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-32-ec6f64616f8f>:3 in <cell line: 3>                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:140 in __call__                 │
│                                                                                                  │
│   137 │   │   │   )                                                                              │
│   138 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   139 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 140 │   │   │   raise e                                                                        │
│   141 │   │   run_manager.on_chain_end(outputs)                                                  │
│   142 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   143                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:134 in __call__                 │
│                                                                                                  │
│   131 │   │   )                                                                                  │
│   132 │   │   try:                                                                               │
│   133 │   │   │   outputs = (                                                                    │
│ ❱ 134 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   135 │   │   │   │   if new_arg_supported                                                       │
│   136 │   │   │   │   else self._call(inputs)                                                    │
│   137 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/retrieval_qa/base.py:120 in _call       │
│                                                                                                  │
│   117 │   │   question = inputs[self.input_key]                                                  │
│   118 │   │                                                                                      │
│   119 │   │   docs = self._get_docs(question)                                                    │
│ ❱ 120 │   │   answer = self.combine_documents_chain.run(                                         │
│   121 │   │   │   input_documents=docs, question=question, callbacks=_run_manager.get_child()    │
│   122 │   │   )                                                                                  │
│   123                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:239 in run                      │
│                                                                                                  │
│   236 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   237 │   │                                                                                      │
│   238 │   │   if kwargs and not args:                                                            │
│ ❱ 239 │   │   │   return self(kwargs, callbacks=callbacks)[self.output_keys[0]]                  │
│   240 │   │                                                

In [ ]:
llm_response

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
chain = load_qa_with_sources_chain(hf_pipeline, chain_type="stuff")

response = chain({"input_documents": documents, "question": query}, return_only_outputs=True)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1080: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-38-50565dcba2c1>:4 in <cell line: 4>                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:140 in __call__                 │
│                                                                                                  │
│   137 │   │   │   )                                                                              │
│   138 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   139 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 140 │   │   │   raise e                                                                        │
│   141 │   │   run_manager.on_chain_end(outputs)                                                  │
│   142 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   143                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:134 in __call__                 │
│                                                                                                  │
│   131 │   │   )                                                                                  │
│   132 │   │   try:                                                                               │
│   133 │   │   │   outputs = (                                                                    │
│ ❱ 134 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   135 │   │   │   │   if new_arg_supported                                                       │
│   136 │   │   │   │   else self._call(inputs)                                                    │
│   137 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/combine_documents/base.py:84 in _call   │
│                                                                                                  │
│    81 │   │   docs = inputs[self.input_key]                                                      │
│    82 │   │   # Other keys are assumed to be needed for LLM prediction                           │
│    83 │   │   other_keys = {k: v for k, v in inputs.items() if k != self.input_key}              │
│ ❱  84 │   │   output, extra_return_dict = self.combine_docs(                                     │
│    85 │   │   │   docs, callbacks=_run_manager.get_child(), **other_keys                         │
│    86 │   │   )                                                                                  │
│    87 │   │   extra_return_dict[self.output_key] = output                                        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/combine_documents/stuff.py:87 in        │
│ combine_docs                                                                                     │
│                                                                                                  │
│    84 │   │   """Stuff all documents into one prompt and pass to LLM."""                         │
│    85 │   │   inputs = self._get_inputs(docs, **kwargs)                                          │
│    86 │   │   # Call predict on the LLM.                                                         │
│ ❱  87 │   │   return self.llm_chain.predict(callbacks=callb

In [ ]:
docsearch = Chroma.from_texts(documents, embedding)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-42-df5bbc89578f>:1 in <cell line: 1>                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/vectorstores/chroma.py:405 in from_texts       │
│                                                                                                  │
│   402 │   │   │   client_settings=client_settings,                                               │
│   403 │   │   │   client=client,                                                                 │
│   404 │   │   )                                                                                  │
│ ❱ 405 │   │   chroma_collection.add_texts(texts=texts, metadatas=metadatas, ids=ids)             │
│   406 │   │   return chroma_collection                                                           │
│   407 │                                                                                          │
│   408 │   @classmethod                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/vectorstores/chroma.py:159 in add_texts        │
│                                                                                                  │
│   156 │   │   │   ids = [str(uuid.uuid1()) for _ in texts]                                       │
│   157 │   │   embeddings = None                                                                  │
│   158 │   │   if self._embedding_function is not None:                                           │
│ ❱ 159 │   │   │   embeddings = self._embedding_function.embed_documents(list(texts))             │
│   160 │   │   self._collection.add(                                                              │
│   161 │   │   │   metadatas=metadatas, embeddings=embeddings, documents=texts, ids=ids           │
│   162 │   │   )                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/embeddings/huggingface.py:77 in                │
│ embed_documents                                                                                  │
│                                                                                                  │
│    74 │   │   Returns:                                                                           │
│    75 │   │   │   List of embeddings, one for each text.                                         │
│    76 │   │   """                                                                                │
│ ❱  77 │   │   texts = list(map(lambda x: x.replace("\n", " "), texts))                           │
│    78 │   │   embeddings = self.client.encode(texts, **self.encode_kwargs)                       │
│    79 │   │   return embeddings.tolist()                                                         │
│    80                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/embeddings/huggingface.py:77 in <lambda>       │
│                                                                                                  │
│    74 │   │   Returns:                                                                           │
│    75 │   │   │   List of embeddings, one for each text.                                         │
│    76 │   │   """                                                                                │
│ ❱  77 │   │   texts = list(map(lambda x: x.replace("\n", " 

# Notes for next version:
## Fix Token indices sequence length Issue
Reference: 
[Token indices sequence length Issue](https://stackoverflow.com/questions/68850172/token-indices-sequence-length-issue)
